# Aim

In step four, I'd like to address expanding the phylogeny with more R2R3 MYB subfamiles to address some open questions: 
* Can I add a subfamily label to the other Azfi sequences I found before with a custom hmm.
* Can I infer the phylogeny to more confidently/explicitly draw the Salvinales VII branch as a part of the remaining VII branch by including other subfamiles and an outgroup

In a later notebook, I'll address adding extra data like: 
Next steps: 

* Can I add intron/exon structures as a second line of evidence supporting the different subfamilies
* Finally, I'd like to log2 expressen/fold change to the Azfi sequences; to illustrate how these subfamilies change expression upon transistion to sexual reproduction at least in _Azolla_

Since retrieving sequences from the different databases is quite a pain, 
I'll first retrieve only NCBI listed sequences (which I can retrieve in batch).
Subsequently, I may reitterate in this notebook, expanding on that analysis with sequences from other genome databases.

Genes in new list files in the `data` directory were retrieved from NCBI. 
Acc. nr.s from the J&R study were mixed protein and gene id's, so batch retrieval wasn't possible.
Fasta files with both acc nr's were created in the same directory.

# prepare data

Combining and linearising fasta files.

In [6]:
for i in data/*fasta
do  inseq=$(echo $i | cut -d '/' -f 2 | cut -d '.' -f 1)
    cat data/$inseq.fasta \
     | awk '/^>/ {printf("%s%s\n",(N>0?"\n":""),$0);N++;next;} {printf("%s",$0);} END {printf("\n");}' \
     > data/"$inseq"_linear.fasta
done

In [3]:
ls data/*_linear.fasta

data/ARP_sequences_linear.fasta
data/Azfi-mybs-subfamVI-suspects_linear.fasta
data/Azfi-v1-MYB-sequences_linear.fasta
data/Azfi-v1-MYB-sequences_linear_linear.fasta
data/CDC5-outgroup_sequences_linear.fasta
data/combi_sequences_linear.fasta
data/combi_sequences_linear_linear.fasta
data/combi-VI-VII-Azfisuspects_linear.fasta
data/III_sequences_linear.fasta
data/II_sequences_linear.fasta
data/I_sequences_linear.fasta
data/IV_sequences_linear.fasta
data/MYB33_ARATH_linear.fasta
data/MYB33_ARATH_linear_linear.fasta
data/R1R2R3_sequences_linear.fasta
data/VIII_sequences_linear.fasta
data/VII_sequences_linear.fasta
data/VII_sequences_linear_linear.fasta
data/VI_sequences_linear.fasta
data/VI_sequences_linear_linear.fasta


In [7]:
cat data/CDC5-outgroup_sequences_linear.fasta \
    data/I_sequences_linear.fasta    \
    data/II_sequences_linear.fasta   \
    data/III_sequences_linear.fasta  \
    data/IV_sequences_linear.fasta   \
    data/V_sequences_linear.fasta    \
    data/VI_sequences_linear.fasta   \
    data/VII_sequences_linear.fasta  \
    data/VIII_sequences_linear.fasta \
    data/ARP_sequences_linear.fasta  \
    data/Azfi-mybs-subfamVI-suspects_linear.fasta \
    data/R1R2R3_sequences_linear.fasta \
    data/MYB33_ARATH_linear.fasta \
    > data/combi-I-to-VII-Azfi_sequences_linear.fasta
    

In [16]:
conda activate phylogenetics
if    [ ! -d ./data/alignments_raw/ ]
then  mkdir  ./data/alignments_raw
fi
for   i in data/combi*sequences_linear.fasta
do    inseq=$(echo $i | cut -d '/' -f 2 | cut -d '.' -f 1)
      if    [ ! -f "./data/alignments_raw/$inseq"_aligned-mafft-linsi.fasta ]
      then  echo "aligning $inseq"
            linsi --thread $(nproc) data/$inseq.fasta > ./data/alignments_raw/"$inseq"_aligned-mafft-linsi.fasta \
                                                      2> ./data/alignments_raw/"$inseq"_aligned-mafft-linsi.log
      fi
done
conda deactivate

(phylogenetics) (phylogenetics) aligning combi-I-to-VII-Azfi_sequences_linear
(phylogenetics) 

In [21]:
conda activate jalview
for   i in data/alignments_raw/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null
      fi
done
conda deactivate

(jalview) (jalview) 

The alignment looks like this:
![](./data/alignments_raw/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi.png)

It is even more gappy than the alignments we've seen before, but that stands to reason since a lot more of sequence diversity is covered in this MSA. 
After trimming this will look just fine I'm sure.

In [29]:
inseq=combi-I-to-VIII-Azfi_sequences_linear

In [30]:
conda activate phylogenetics
if    [ ! -d data/alignments_trimmed ]
then  mkdir  data/alignments_trimmed 
fi

# define appendix only once here:
trimappendix='trim-gt4'

for a in "data/alignments_raw/$inseq"_aligned*.fasta
do  appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    if    [ ! -f data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta ]
    then  echo "trimming alignment $a"
          sed -i 's/ /_/g' $a
          trimal -in $a   \
                 -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta \
                 -gt .4 \
                 -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html
    fi
done
conda deactivate

(phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) trimming alignment data/alignments_raw/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi.fasta
(phylogenetics) 

In [31]:
conda activate jalview
for   i in data/alignments_trimmed/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null
      fi
done
conda deactivate

(jalview) (jalview) 

The alignment looks like this:
![](./data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt4.png)

I can certainly make a tree from this alignment, first let's try a quick one:

# Tree inference

In [32]:
conda activate phylogenetics
for a in data/alignments_trimmed/"$inseq"_aligned*gt[46].fasta
do  #iqpendix='iqtree-b100'
    iqpendix='iqtree-bb2000-alrt2000'

    echo "making a tree of file $a"
    echo "The first lines of alignment $a look like this"
    head $a

    file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    
    if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
    then echo "Making a directory $file_appendix to store trees (name based on alignment filename)"
         mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
    fi

    iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
    if   [ ! -f "$iqprefix".tree ]
    then nice iqtree -s $a \
                     -m MFP \
                     -bb 2000 -alrt 2000 \
                     -nt AUTO \
                     -ntmax $(nproc)  \
                     -pre  "$iqprefix" \
                     2>   "$iqprefix".stderr \
                      >    "$iqprefix".stdout
    cat "$iqprefix".log | mail -s "IQtree_run $a" laura
    fi
done
conda deactivate

(phylogenetics) making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt4.fasta
The first lines of alignment data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt4.fasta look like this
>CDC5_XM_024677933.1_XP_024533701.1_transcription_factor_MYB61-like_[Selaginella_moellendorffii]
M--SRREQPDDHE------TSSSSIVRKGLWSPKEDELLLNFILQHGCGNWTTVPKLAGL
QRSGKSCRLRWMNYLRPDLKRGKFSDEENQKLIELHGLVGNRWAYIASQLPGRTDNDVKN
QWNSRIRNKAL-----------AHSPVSERSDPPPSPPT-PEIPDPAGEEFRDTHLLQSD
PAEEPASSPAAKSSA---------------------------------------------
--SASLQSDG--------------------------------------------------
------------------------------SPMMDG------------------------
----------------------------DL---------------------------GCV
VWQF--
>CDC5_XM_024686040.1_XP_024541808.1_cell_division_cycle_5-like_protein
Making a directory aligned-mafft-linsi_trim-gt4 to store trees (name based on alignment filename)
(phylogenetics) 

This [initial tree](https://itol.embl.de/tree/1312115964244391596537363#) puts some sequences at odd places. 
I'm wondering if I have a too gappy alignment here, like I already suspected for earlier alignments.
Alternativelly, I may try to align sequences with the einsi method in stead of the linsi method, since the big conserved domain is quite gappy and linsi may not align conserved sites outside of the main domain propperly.